In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('sentence_bank.sqlite3')
cursor = conn.cursor()
cursor2 = conn.cursor()

Load sentences for storage

In [4]:
sentences = [[]]

In [5]:
with open('ner.txt', 'r') as f:
    for line in f:
        if line.strip() == '':
            sentences.append([])
        else:
            sentences[-1].append(line.split())
            
sentences.pop()
pass

In [6]:
len(sentences) == 3655

True

In [7]:
for sentence in sentences:
    assert len(sentence) > 0
    for word, tag in sentence:
        assert tag in ['D', 'T', 'O']

In [9]:
D_count = 0
T_count = 0
O_count = 0
D_scount = 0
T_scount = 0
neither_scount = 0

for sentence in sentences:
    D_found = False
    T_found = False
    
    for word, tag in sentence:
        if tag == 'D':
            D_count += 1
            D_found = True
        elif tag == 'T':
            T_count += 1
            T_found = True
        else:
            O_count += 1
    
    if D_found:
        D_scount += 1
    if T_found:
        T_scount += 1
    if not D_found and not T_found:
        neither_scount += 1
    
print 'D:', D_count, 'occurrences in', D_scount, 'sentences'
print 'T:', T_count, 'occurrences in', T_scount, 'sentences'
print 'O:', O_count, 'occurrences'
print neither_scount, 'sentences have neither D nor T, that\'s', '%.2f%%!' % (float(neither_scount)/36.55)

D: 4889 occurrences in 1644 sentences
T: 3821 occurrences in 1183 sentences
O: 55810 occurrences
1843 sentences have neither D nor T, that's 50.42%!


In [10]:
O_count/float(D_count + T_count + O_count)

0.8650030998140111

In [ ]:
cursor.execute('DROP TABLE sentences;')
cursor.execute('DROP TABLE words;')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS sentences (
   id INTEGER PRIMARY KEY AUTOINCREMENT
);''')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS words (
   sentence_id INTEGER,
   word_id INTEGER,
   word VARCHAR,
   tag CHARACTER(1),
   FOREIGN KEY(sentence_id) REFERENCES sentence(id),
   PRIMARY KEY(sentence_id, word_id)
);''')

In [ ]:
for sentence in sentences:
    cursor.execute('INSERT INTO sentences (id) VALUES (NULL);')
    sentence_id = cursor.lastrowid
    
    for i, word_tag in enumerate(sentence):
        word, tag = word_tag
        assert tag in ['D', 'T', 'O']
        
        cursor.execute('INSERT INTO words (sentence_id, word_id, word, tag) VALUES (?, ?, ?, ?);',
                       (sentence_id, i, word.decode('utf8', 'ignore'), tag))

In [ ]:
print '[Actual]'
print 'Sentences:', sentence_id
print 'Words:    ', cursor.lastrowid

In [ ]:
print '[Expected]'
print 'Sentences:', len(sentences)
print 'Words:    ', sum(map(len, sentences))

In [ ]:
conn.commit()

POS Tagging

In [ ]:
import nltk

In [ ]:
cursor.execute('DELETE FROM pos_tags;')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS pos_tags (
   sentence_id INTEGER,
   word_id INTEGER,
   pos_tag VARCHAR,
   FOREIGN KEY(sentence_id, word_id) REFERENCES words(sentence_id, word_id),
   PRIMARY KEY(sentence_id, word_id)
);''')

In [ ]:
cursor.execute('SELECT * FROM sentences;')

for row in cursor:
    sentence_id = row[0]
    cursor2.execute('SELECT word_id, word FROM words WHERE sentence_id = ? ORDER BY word_id', (sentence_id,))
    
    sentence = []
    
    for i, row2 in enumerate(cursor2):
        word_id, word = row2
        assert i == word_id
        sentence.append(word)
    
    tags = nltk.pos_tag(sentence)
    
    for i, word_tag in enumerate(tags):
        word, pos_tag = word_tag
        cursor2.execute('INSERT INTO pos_tags (sentence_id, word_id, pos_tag) VALUES (?, ?, ?);',
                       (sentence_id, i, pos_tag))

In [3]:
for row in cursor.execute('SELECT COUNT(DISTINCT pos_tag) FROM pos_tags;'):
    print row

(43,)


In [ ]:
conn.commit()

WordNet

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
def get_hyponyms(concept, stringify=True):
    concept = wn.synset(concept)
    hyponyms = set([i for i in concept.closure(lambda s:s.hyponyms())])
    
    if stringify:
        hyponyms = set([str(hyponym)[8:-7].replace('_', ' ') for hyponym in hyponyms])
    
    assert type(hyponyms) == set
    return hyponyms

In [ ]:
cursor.execute('DELETE FROM wordnet_tags;')

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS wordnet_tags (
   sentence_id INTEGER,
   word_id INTEGER,
   wordnet_tag VARCHAR,
   FOREIGN KEY(sentence_id, word_id) REFERENCES words(sentence_id, word_id),
   PRIMARY KEY(sentence_id, word_id, wordnet_tag)
);''')

In [ ]:
concepts_of_interest = ['body_part.n.01', 'medicine.n.01', 'treatment.n.01', 'disease.n.01', 'chemical.n.01']
hyponyms = dict([(concept, get_hyponyms(concept)) for concept in concepts_of_interest])

In [ ]:
cursor.execute('SELECT * FROM sentences;')

for row in cursor:
    sentence_id = row[0]
    cursor2.execute('SELECT word_id, word FROM words WHERE sentence_id = ? ORDER BY word_id', (sentence_id,))
    
    sentence = []
    
    for i, row2 in enumerate(cursor2):
        word_id, word = row2
        assert i == word_id
        sentence.append(word)
    
    sentence_lower = [w.lower() for w in sentence]
    tags = []

    for concept in concepts_of_interest:
        for hyponym in hyponyms[concept]:
            hyponym = hyponym.split()

            for i in range(len(sentence) - len(hyponym) + 1):
                if sentence_lower[i:i+len(hyponym)] == hyponym:
                    tags += [(i, i+len(hyponym), concept)]
    
    for tag in tags:
        start, end, wordnet_tag = tag

        for word_id in range(start, end):
            cursor2.execute('INSERT OR IGNORE INTO wordnet_tags (sentence_id, word_id, wordnet_tag) VALUES (?, ?, ?);',
                       (sentence_id, word_id, wordnet_tag))

In [ ]:
for concept in concepts_of_interest:
    print '\033[1m[%s:' % (concept,),
    
    for row in cursor.execute('SELECT COUNT(*) FROM wordnet_tags WHERE wordnet_tag = ?;', (concept,)):
        print row[0], '(total)]', '\033[0;0m'
        
    # Check the corresponding labels of tags!
    for row in cursor2.execute('''
    SELECT tag, COUNT(*) FROM words INNER JOIN wordnet_tags
    ON words.sentence_id = wordnet_tags.sentence_id
    AND words.word_id = wordnet_tags.word_id
    WHERE wordnet_tag = ?
    GROUP BY tag;
    ''', (concept,)):
        print '%s: %d' % row
        
    print ''

In [ ]:
conn.commit()

MeSH dataset

In [3]:
def fix_comma(word):
    if ',' in word and ',' not in word[word.index(',') + 1:]:
        return (word[word.index(',') + 1:] + ' ' + word[:word.index(',')]).strip()
    else:
        return word

In [7]:
qualifiers = dict()

with open('q2018.bin', 'r') as f:
    subject_heading = None
    
    for line in f:
        if line.strip() in ['', '*NEWRECORD']:
            subject_heading = None
            continue
        else:
            key, label = line.split(' = ', 1)
        
        if key == 'SH':
            subject_heading = fix_comma(label.strip().lower())
            qualifiers[subject_heading] = set()
        elif key == 'QX':
            qualifiers[subject_heading].add(fix_comma(label.split('|')[0].strip().lower()))

In [8]:
descriptors = dict()

with open('d2018.bin', 'r') as f:
    subject_heading = None
    
    for line in f:
        if line.strip() in ['', '*NEWRECORD']:
            subject_heading = None
            continue
        else:
            key, label = line.split(' = ', 1)
        
        if key == 'MH':
            subject_heading = fix_comma(label.strip().lower())
            descriptors[subject_heading] = set()
        elif key == 'ENTRY':
            descriptors[subject_heading].add(fix_comma(label.split('|')[0].strip().lower()))

In [9]:
all_qualifiers  = [[k] + list(v) for k,v in qualifiers.iteritems() ]
all_descriptors = [[k] + list(v) for k,v in descriptors.iteritems()]

In [10]:
all_qualifiers  = set().union(*all_qualifiers)
all_descriptors = set().union(*all_descriptors)

In [11]:
reverse_qualifier_map  = dict(set().union(*[[(v1, k) for v1 in v] + [(k, k)] for k, v in qualifiers.iteritems()]))
reverse_descriptor_map = dict(set().union(*[[(v1, k) for v1 in v] + [(k, k)] for k, v in descriptors.iteritems()]))

In [12]:
cursor.execute('DROP TABLE mesh_tags;')

In [13]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS mesh_tags (
   sentence_id INTEGER,
   word_id INTEGER,
   mesh_type VARCHAR,
   mesh_keyword VARCHAR,
   FOREIGN KEY(sentence_id, word_id) REFERENCES words(sentence_id, word_id),
   PRIMARY KEY(sentence_id, word_id, mesh_type, mesh_keyword)
);''')

In [26]:
num_in = 0
to_remove = set()

for word in all_descriptors:
    if '0' in word or '1' in word or '2' in word or '3' in word or '4' in word or '(' in word or \
       '5' in word or '6' in word or '7' in word or '8' in word or '9' in word or '%' in word:
        to_remove.add(word)

for word in to_remove:
    all_descriptors.remove(word)

print len(all_descriptors)

117241


In [27]:
cursor.execute('SELECT * FROM sentences;')

for row in cursor:
    sentence_id = row[0]
    cursor2.execute('SELECT word_id, word FROM words WHERE sentence_id = ? ORDER BY word_id', (sentence_id,))
    
    sentence = []
    
    for i, row2 in enumerate(cursor2):
        word_id, word = row2
        assert i == word_id
        sentence.append(word)
    
    sentence_lower = [w.lower() for w in sentence]
    tags = []

    for qualifier in all_qualifiers:
        og_qualifier = qualifier
        qualifier = qualifier.split()

        for i in range(len(sentence) - len(qualifier) + 1):
            if sentence_lower[i:i+len(qualifier)] == qualifier:
                tags += [(i, i+len(qualifier), 'qualifier', reverse_qualifier_map[og_qualifier])]
                    
    for tag in tags:
        start, end, mesh_type, mesh_tag = tag

        for word_id in range(start, end):
            cursor2.execute('INSERT OR IGNORE INTO mesh_tags (sentence_id, word_id, mesh_type, mesh_keyword) VALUES (?, ?, ?, ?);',
                       (sentence_id, word_id, mesh_type, mesh_tag))

In [29]:
conn.commit()

And... we're done!

In [30]:
conn.close()